In [3]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from ckonlpy.tag import Twitter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

In [4]:
def keyword_part(doc,stopword) :
    doc = pd.read_csv(doc)
    doc = list(doc["checked"])
    # doc = doc.split('\n')
    text = pd.read_csv(stopword)
    text_list = list(np.array(text['불용어'].tolist()))
    twitter = Twitter()
    twitter.add_dictionary(['암묵지','창출','유즈 케이스','유즈케이스','내재화','해결','고려사항','장기적','단기적','사이언티스트','전처리','이상치','결측치','학습용 데이터','인공지능','텍스트 마이닝','메타버스','키워드','시사점','가속화','선도국','고품질','적극적','투자','불확실성','감소','중장기','장년층','노년층','상대적','최소화','디지털 리터러시 교육','세대별','맞춤형','어디서나','특성','긍정적','시공간'], 'Noun')
    key_part = []
    tokenized_nouns = []
    tokenized_nouns_11 = []
    result = []
    candidates = []
    # candidates_list = []
    
    for i in range(0,len(doc)) :
        
        key_part.append(twitter.pos(doc[i]))
        tokenized_nouns.append(','.join([word[0] for word in key_part[i] if word[1] == 'Noun']))
        tokenized_nouns_11.append([tokenized_nouns[i]])
        result.append([word for word in tokenized_nouns_11[i] if not word in text_list])
    
        n_gram_range = (0, 3)
        
        count = CountVectorizer(ngram_range=n_gram_range).fit(result[i])
        temp_ = list(count.get_feature_names_out())
        candidates.append(temp_)
    
    result = []
    for i in range(len(candidates)):
        result += candidates[i]
        
    model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    doc_embedding = model.encode([doc_2])
    candidate_embeddings = model.encode(result)
    
    top_n = 5
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [result[index] for index in distances.argsort()[0][-top_n:]]
    
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

# 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                        candidate_embeddings)

# 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-30:])
    words_vals = [result[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
                candidate = combination
                min_sim = sim

    return [words_vals[idx] for idx in candidate]
    
        